# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

# План выполнения проекта

* 1. Откроем файл с данными и изучим его.

* 2. Разделим исходные данные на обучающую, валидационную и тестовую выборки.

* 3. Исследуем качество разных моделей, меняя гиперпараметры.

* 4. Проверить качество модели на тестовой выборке.

* 5.  Проверим модели на вменяемость.


# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# Импортируем библиотеки

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

# Вводим переменные

In [5]:
RANDOM_STATE = 12345 #Зафиксируем псевдослучайность

## Откроем и изучим файл.

In [2]:
data_1 = 'users_behavior.csv'
data_2 = 'https://code.s3.yandex.net/datasets/users_behavior.csv'

if os.path.exists(data_1):
    users_data = pd.read_csv(data_1)
elif os.path.exists(data_2):
    users_data = pd.read_csv(data_2)
else:
    print('Something is wrong')

Выведем основную информацию о датафрейме с помощью метода info():

In [3]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Выведем первые 10 строчек датафрейма на экран:

In [4]:
users_data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Выводы после изучения данных.

По предварительному просмотру основной информации данных можно отметить следующее:

Предобработка данных не понадобится так как мы уже её сделали в предыдущем проекте.

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разобъем в соотношении 3:1:1:
* Обучающая выборка (60%);
* Валидационная выборка (20%);
* Тестовая выборка (20%).

## Разделим исходные данные на обучающую, валидационную и тестовую выборки.

Разделим исходные данные на 3 выборки:
* `train_users_data` - обучающая выборка;
* `validate_users_data` - валидационная выборка;
* `test_users_data` - тестовая выборка.

In [6]:
train_users_data, validate_test_users_data = train_test_split(users_data, test_size=0.4, random_state=RANDOM_STATE)

In [7]:
validate_users_data, test_users_data = train_test_split(validate_test_users_data, test_size=0.5, random_state=RANDOM_STATE)

Проверим размер разбитых выборок:

In [8]:
def check_data_split(splited_df, df = users_data):
    split_val = round((splited_df.shape[0] / df.shape[0]) * 100, 2)
    return f'Размер данных разбитой выборки от исходных данных составляет: {split_val} %'
    

In [9]:
check_data_split(train_users_data)

'Размер данных разбитой выборки от исходных данных составляет: 59.99 %'

In [10]:
check_data_split(validate_users_data)

'Размер данных разбитой выборки от исходных данных составляет: 20.01 %'

In [11]:
check_data_split(test_users_data)

'Размер данных разбитой выборки от исходных данных составляет: 20.01 %'

### Вывод после разделения исходнях данных.

Провели разделение исходных данных на обучающую, валидационную и тестовую выборки.

Проверили корректность разделения данных.

##  Исследуем качество разных моделей, меняя гиперпараметры.

Исследуем качество разных моделей, а именно:
* Решающее дерево;
* Случайный лес;
* Логическая регрессия.
Логическую регрессию используем из-за параметра, который необходимо предсказать (он категориальный).

Обучающий и валидационный набор данных сохраним в отдельных переменных features (признаки) и target (целевой признак).

In [12]:
features_train = train_users_data.drop('is_ultra', axis=1)
target_train = train_users_data['is_ultra']
features_valid = validate_users_data.drop('is_ultra', axis=1)
target_valid = validate_users_data['is_ultra']

In [13]:
best_result = 0
best_est = 0
best_depth = 0
best_model_name = None
best_result_decision_tree = 0
best_result_random_forest = 0 
best_result_logistic_regression = 0

### Решающее дерево.

Меняя гиперпараметры, а cамый важный гиперпараметр решающего дерева — `max_depth` глубину дерева. Обучим модель на обучающей выборке, получим предсказания модели на валидационной выборке и посчитаем качество модели.

Зафиксируем псевдослучайность параметром `random_state`.

In [14]:
for depth in range(1, 10):
    model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_decision_tree.fit(features_train, target_train)
    predictions = model_decision_tree.predict(features_valid)
    result_decision_tree = accuracy_score(target_valid, predictions)
    if result_decision_tree > best_result_decision_tree:
        best_result_decision_tree = result_decision_tree
        best_depth = depth
        if result_decision_tree > best_result:
            best_result = result_decision_tree
            best_model_name = 'DecisionTreeClassifier'

In [15]:
print("Accuracy решающего дерева:", round(best_result_decision_tree, 3))
print("Глубина дерева при лучшей accuracy модели:", best_depth)

Accuracy решающего дерева: 0.785
Глубина дерева при лучшей accuracy модели: 3


### Случайный лес.

Меняя гиперпараметры, пропишем `n_estimators`. Чем больше деревьев, тем дольше модель будет учиться, но результат станет лучше (и наоборот). Обучим модель на обучающей выборке, получим предсказания модели на валидационной выборке и посчитаем качество модели.

Зафиксируем псевдослучайность параметром `random_state`.

In [16]:
for est in range(1, 25):
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_random_forest.fit(features_train, target_train)
    result_random_forest = model_random_forest.score(features_valid, target_valid)
    if result_random_forest > best_result_random_forest:
        best_result_random_forest = result_random_forest
        best_est = est
        if result_random_forest > best_result:
            best_result = result_random_forest
            best_model_name = 'RandomForestClassifier'

In [17]:
print("Accuracy случайного леса:", round(best_result_random_forest, 3))
print("Количество деревьев при лучшей accuracy модели:", best_est)

Accuracy случайного леса: 0.795
Количество деревьев при лучшей accuracy модели: 23


### Логическая регрессия.

Запишите модель в переменной, указав гиперпараметры: `solver` = `'lbfgs'` и `max_iter`. Первый гиперпараметр позволяет выбрать алгоритм, который будет строить модель. Алгоритм `'lbfgs'` — один из самых распространённых. Он подходит для большинства задач. Гиперпараметром `max_iter` задаётся максимальное количество итераций обучения. Значение этого параметра по умолчанию равно 100, но в некоторых случаях понадобится больше итераций.

Зафиксируем псевдослучайность параметром `random_state`.

In [18]:
for max_iter in range(1, 200):
    model_logistic_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=max_iter)
    model_logistic_regression.fit(features_train, target_train)
    result_logistic_regression = model_logistic_regression.score(features_valid, target_valid)
    if result_logistic_regression > best_result_logistic_regression:
        best_result_logistic_regression = result_logistic_regression
        best_max_iter = max_iter
        if result_logistic_regression > best_result:
            best_result = result_logistic_regression
            best_model_name = 'LogisticRegression'

In [19]:
print("Accuracy логической регрессии:", round(best_result_logistic_regression, 3))
print("Количество итераций обучения при лучшей accuracy модели:", best_max_iter)

Accuracy логической регрессии: 0.711
Количество итераций обучения при лучшей accuracy модели: 39


Выведем название лучшей модели и значение её accuracy: 

In [20]:
print("Модель:", best_model_name)
print(round(best_result,3))

Модель: RandomForestClassifier
0.795


### Вывод после исследования разных моделей.

Исследовали качество разных моделей, меняя гиперпараметры, а именно:
* Решающее дерево:
  * Accuracy решающего дерева: 0.785;
  * Глубина дерева при лучшей accuracy модели: 3.
* Случайный лес:
  * Accuracy случайного леса: 0.795;
  * Количество деревьев при лучшей accuracy модели: 23.
* Логическая регрессия:
  * Accuracy логической регрессии: 0.711;
  * Количество итераций обучения при лучшей accuracy модели: 39.
  
Лучшая модель и её accuracy:
 RandomForestClassifier
 0.795

## Проверим качество модели на тестовой выборке.

Тестовый набор данных сохраним в отдельных переменных features (признаки) и target (целевой признак):

In [21]:
features_test = test_users_data.drop('is_ultra', axis=1)
target_test = test_users_data['is_ultra']

Получим предсказания модели на тестовой выборке:

In [22]:
predictions_test = model_random_forest.predict(features_test)
accuracy_score = round(accuracy_score(target_test, predictions_test), 3)

In [23]:
print("Accuracy случайного леса на тестовой выборке:", accuracy_score)

Accuracy случайного леса на тестовой выборке: 0.782


###  Вывод после проверки качества модели на тестовой выборке:

Проверили качество модели на тестовой выборке и получили accuracy 0.782

## Проверим модели на вменяемость.

Для проверки используем модель DummyClassifier:

In [24]:
model_dummy_classifier = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dummy_classifier.fit(features_train, target_train)
result_dc_valid = model_dummy_classifier.score(features_valid, target_valid)
result_dc_test = model_dummy_classifier.score(features_test, target_test)

In [25]:
print('Значение accuracy DummyClassifier по валидационной выборке:', round(result_dc_valid, 3))
print('Значение accuracy DummyClassifier по тестовой выборке:', round(result_dc_test, 3))

Значение accuracy DummyClassifier по валидационной выборке: 0.706
Значение accuracy DummyClassifier по тестовой выборке: 0.684


### Вывод после дополнительного задания:

Проверили модели на вменяемость используя модель DummyClassifier.
Значения accuracy модели DummyClassifier (0.706 и 0.684) ниже, чем у выбранной нами модели (0.782), значит наша модель адекватна и эффективна.

## Общий вывод

По результатам исследования можно отметить сделать следующие выводы:

1.Проведено исследование с целью разработки модели, позволяющей наиболее точно предложить пользователям мобильной связи тарифы «Смарт» или «Ультра».

2.Модель построена на основании поведения клиентов, уже перешедших на эти тарифы.

3.Были построены 3 модели:

* дерево решений `Decision Tree Classifier`;
* случайный лес `RandomForestClassifier`;
* логистическая регрессия `Logistic Regression`.

4.По результатам сравнения была выбрана лучшая - 'случайный лес' с гиперпараметрами n_estimators равным `5`. Значение accuracy = `0.782`.

5.Лучшая модель проверена на тестовых объектах, не участвующих в обучении и валидации модели. Значение accuracy = `0.788`.

7.Для проверки на адекватность выбранной нами модели 'случайный лес' ( RandomForestClassifier ) использовали модель DummyClassifier, которая показала результат accuracy хуже, чем результат нашей лучшей модели.

8.Результаты исследования позволят специалистам выбрать наиболее качественную модель, которая подберет клиентам новый тариф.